In [25]:
import v20
import pandas as pd
import numpy as np
import json
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import os
import mplfinance as mpf
import openai
import base64
import requests
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# OANDA API configuration using environment variables
api = v20.Context(
    hostname='api-fxpractice.oanda.com',
    port=443,
    ssl=True,
    token=os.getenv('OANDA_API_TOKEN'),
    datetime_format='RFC3339'
)
access_token = os.getenv('OANDA_API_TOKEN')

# Set OpenAI API key from environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')


# Set the parameters for real-time data fetching and processing
granularity = 'M5'
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '5 minutes'
base_dir = f'{pair}/{timeframe}'
window_size = 576  # Adjust window size for detecting single patterns
step_size = 5      # Adjust step size accordingly


In [26]:
import logging
import pandas as pd
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory

# Configure logging
logging.basicConfig(level=logging.INFO)

from datetime import datetime, timedelta, timezone

def fetch_forex_data(from_date, to_date, granularity, instrument):
    client = API(access_token=access_token, environment="practice")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = client.request(request)
            if response:
                for candle in response.get('candles'):
                    # Correctly handle time format for the OANDA API
                    time = candle.get('time').split('.')[0] + 'Z'  # Remove milliseconds and add 'Z'
                    rec = {
                        'time': time,
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    return pd.DataFrame(data)


In [27]:


# # Example usage with variables for the date range
# end_time = (datetime.now(timezone.utc) - timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%SZ')
# start_time = (datetime.now(timezone.utc) - timedelta(days=4)).strftime('%Y-%m-%dT%H:%M:%SZ')
# print(start_time)
# print(end_time)
# logging.info(f"Fetching data from {start_time} to {end_time}")

# prices = fetch_forex_data(start_time, end_time, 'M15', 'EUR_USD')
# logging.info(prices.head())

# # prices = fetch_forex_data('2024-05-14T00:00:00Z', '2024-05-17T00:00:00Z', 'M15', 'EUR_USD')
# print(prices)

In [28]:
import mplfinance as mpf

# Plot candlestick chart
def plot_candlestick_chart(df, filename, response_text=None):
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')
    addplots = []
    if 'SMA20' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA20'], color='blue'))
    if 'SMA50' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA50'], color='orange'))
    if response_text:
        addplots.append(mpf.make_addplot([None] * len(df), type='text', text=response_text, ax='secondary', secondary_y=False))
    mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, savefig=filename)


In [29]:
# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Analyze image with GPT-4o
def analyze_image_with_gpt4o(filename, chart_count, pair, timeframe, window_data):
    logging.info(f"Analyzing image {filename} with GPT-4o")
    base64_image = encode_image(filename)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    prompt = (
        "Please check this image pattern and give me the list of patterns for EUR/USD 5-minute chart on this chart. "
        "Provide the pattern detection results in JSON format, including the following details:\n"
        "- id\n"
        "- pattern_detected (0 for no pattern, 1 for one pattern, 2 for two patterns, 3 for three patterns, etc.)\n"
        "- pattern name\n"
        "- pattern type\n"
        "- confidence percentage (from 1 to 100)\n"
        "- entry point for trade\n"
        "- take profit\n"
        "- stop loss\n"
        "- best time for exiting order if take profit or stop loss is not achieved\n"
        "- order id\n"
        "- input data (pairs, timeframe, image name)\n"
        "- description (any additional notes or ideas not covered by the other fields)\n\n"
        "If no patterns are detected, include an empty JSON array with 'pattern_detected' set to 0.\n\n"
        "Please note: I do not intend to use this data for trading or any other financial work. Please just double check the chart and provide the correct answer.\n\n"
        "Don't send any description, text, or other response rather than JSON format\n\n"
        "Here is an example of the JSON format:\n\n"
        "[\n"
        "    {\n"
        "        \"id\": 1,\n"
        "        \"pattern_detected\": 1,\n"
        "        \"pattern_name\": \"Double Top\",\n"
        "        \"pattern_type\": \"Reversal\",\n"
        "        \"confidence_percentage\": 90,\n"
        "        \"entry_point\": \"2024-01-12T06:00:00Z\",\n"
        "        \"take_profit\": 1.0800,\n"
        "        \"stop_loss\": 1.0950,\n"
        "        \"best_exit_time\": \"2024-01-13T06:00:00Z\",\n"
        "        \"order_id\": \"ORD123456\",\n"
        "        \"input_data\": {\n"
        "            \"pairs\": \"EUR/USD\",\n"
        "            \"timeframe\": \"5 minutes\",\n"
        "            \"image_name\": \"image1.png\"\n"
        "        },\n"
        "        \"description\": \"Formed after a significant upward trend.\"\n"
        "    },\n"
        "    {\n"
        "        \"id\": 2,\n"
        "        \"pattern_detected\": 2,\n"
        "        \"pattern_name\": \"Head and Shoulders\",\n"
        "        \"pattern_type\": \"Reversal\",\n"
        "        \"confidence_percentage\": 85,\n"
        "        \"entry_point\": \"2024-01-10T04:00:00Z\",\n"
        "        \"take_profit\": 1.0750,\n"
        "        \"stop_loss\": 1.0900,\n"
        "        \"best_exit_time\": \"2024-01-11T04:00:00Z\",\n"
        "        \"order_id\": \"ORD123457\",\n"
        "        \"input_data\": {\n"
        "            \"pairs\": \"EUR/USD\",\n"
        "            \"timeframe\": \"5 minutes\",\n"
        "            \"image_name\": \"image2.png\"\n"
        "        },\n"
        "        \"description\": \"Indicates a possible reversal of the current trend.\"\n"
        "    }\n"
        "]\n"
        "If no patterns are detected:\n"
        "[\n"
        "    {\n"
        "        \"id\": 1,\n"
        "        \"pattern_detected\": 0\n"
        "    }\n"
        "]"
    )
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 300
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    pattern_data = response.json()
    pattern_data['id'] = chart_count
    pattern_data['pair'] = pair
    pattern_data['timeframe'] = timeframe
    if 'input_data' not in pattern_data:
        pattern_data['input_data'] = {}
    pattern_data['input_data']['pairs'] = pair
    pattern_data['input_data']['timeframe'] = timeframe
    pattern_data['input_data']['image_name'] = filename
    return pattern_data

In [30]:
# Function to place an order
def place_order(pattern):
    order_details = {
        'id': pattern['id'],
        'pattern_name': pattern['pattern_name'],
        'pattern_type': pattern['pattern_type'],
        'confidence_percentage': pattern['confidence_percentage'],
        'entry_point': pattern['entry_point'],
        'take_profit': pattern['take_profit'],
        'stop_loss': pattern['stop_loss'],
        'best_exit_time': pattern['best_exit_time'],
        'order_id': pattern['order_id']
    }
    with open('orders_log.json', 'a') as log_file:
        json.dump(order_details, log_file, indent=4)
        log_file.write('\n')
    print(f"Placing order: {order_details}")


In [31]:
import matplotlib.pyplot as plt
import json
import os

# Updated create_moving_window_charts function
import os

def create_moving_window_charts(prices, window_size, step_size, base_dir, start_date, end_date):
    start_index = 0
    end_index = window_size
    
    chart_count = 0
    logging.info(f"Creating moving window charts with window size: {window_size}, len(prices) :{len(prices)} step size: {step_size}")
    while end_index <= len(prices):
        window_data = prices.iloc[start_index:end_index].copy()
        
        year = 2024  # Assuming you want to use a fixed year for the directory structure
        year_dir = os.path.join(base_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)  # Ensure the year directory exists
        
        # Replace colons and hyphens with underscores
        start_date = start_date.replace(':', '_').replace('-', '_')
        end_date = end_date.replace(':', '_').replace('-', '_')

      

        # Ensure proper formatting of the file path
        filename = os.path.join(year_dir, f'{start_date}_to_{end_date}.png')
        filename = filename.replace('\\', '/')
        logging.info(f"Creating filename {filename}: year {year_dir}, {start_date}_to_{end_date}.png")
        
        plot_candlestick_chart(window_data, filename)
        
        result = analyze_image_with_gpt4o(filename, chart_count, pair, timeframe, window_data)
        response_text = json.dumps(result, indent=4)

        # plot_candlestick_chart_with_analysis(window_data, filename, response_text)

        start_index += step_size
        end_index = start_index + window_size
        chart_count += 1


def plot_candlestick_chart_with_analysis(df, filename, analysis_text):
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')
    addplots = []
    if 'SMA20' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA20'], color='blue'))
    if 'SMA50' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA50'], color='orange'))
    
    fig, axes = mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, returnfig=True)
    
    for ax in axes:
        ax.set_title(ax.get_title(), pad=20)  # Increase title padding

    fig.text(0.5, 0.05, analysis_text, ha='center', wrap=True)  # Add analysis text below the plot
    fig.savefig(filename)
    logging.info(f"Saved chart with analysis to: {filename}")
    plt.close(fig)


In [32]:
import logging
import time
from datetime import datetime, timedelta, timezone

# Ensure all your imports and configurations are correct, including the fetch_forex_data function

def fetch_and_process_realtime_data(granularity, instrument, pair, timeframe, base_dir, window_size, step_size):
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
    start_time = (datetime.now(timezone.utc) - timedelta(days=5)).strftime('%Y-%m-%dT%H:%M:%SZ')
    print("start_time1: ", start_time)
    print("end_time1: ", end_time)
    logging.info(f"Fetching initial data from {start_time} to {end_time}")
    initial_prices = fetch_forex_data(start_time, end_time, granularity, instrument)

    if initial_prices.empty:
        logging.error("No initial data fetched. Exiting.")
        return

    create_moving_window_charts(initial_prices, window_size, step_size, base_dir,start_time,end_time)

    while True:
     
        logging.info("Sleeping for 5 minutes before fetching new data")
        time.sleep(300)



# Create base directory if it does not exist
os.makedirs(base_dir, exist_ok=True)

# Run the real-time data fetching and processing
fetch_and_process_realtime_data(granularity, instrument, pair, timeframe, base_dir, window_size, step_size)


INFO:root:Fetching initial data from 2024-05-16T02:33:55Z to 2024-05-21T02:33:55Z
INFO:oandapyV20.oandapyV20:setting up API-client for environment practice
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles


start_time1:  2024-05-16T02:33:55Z
end_time1:  2024-05-21T02:33:55Z


INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:root:Creating moving window charts with window size: 576, len(prices) :831 step size: 5
INFO:root:Creating filename EUR_USD/5 minutes/2024/2024_05_16T02_33_55Z_to_2024_05_21T02_33_55Z.png: year EUR_USD/5 minutes\2024, 2024_05_16T02_33_55Z_to_2024_05_21T02_33_55Z.png
INFO:root:Analyzing image EUR_USD/5 minutes/2024/2024_05_16T02_33_55Z_to_2024_05_21T02_33_55Z.png with GPT-4o
INFO:root:Creating filename EUR_USD/5 minutes/2024/2024_05_16T02_33_55Z_to_2024_05_21T02_33_55Z.png: year EUR_USD/5 minutes\2024, 2024_05_16T02_33_55Z_to_2024_05_21T02_33_55Z.png
INFO:root:Analyzing image EUR_USD/5 minutes/2024/2024_05_16T02_33_55Z_to_2024_05_21T02_33_55Z.png with GPT-4o
INFO:root:Creating filename EUR_USD/5 minutes/2024/2024_05_16T02_33_55Z_to_2024_05_21T02_33_55Z.png: year EUR

KeyboardInterrupt: 